In [1]:
!pip install psycopg2

In [2]:
import psycopg2

import sys
sys.path.append('../../')
from src.pySQL_funcs import pretty_query

with open('../../src/pw') as pw_file:
    pw = pw_file.readline()

In [3]:
conn = psycopg2.connect(pw) #CONTACT ME FOR ACCESS
cur = conn.cursor()

In [4]:
# Example Query to test connection
query = """
SELECT * FROM pokemon LIMIT 10;
"""
pretty_query(cur, query, conn)

,name,type_1,type_2,hp,attack,defense,sp_attack,sp_defense,speed
id,,,,,,,,,
0,Gliscor,Ground,Flying,75,95,125,45,75,95
1,Fletchinder,Fire,Flying,62,73,55,56,52,84
2,Poliwrath,Water,Fighting,90,95,95,70,90,70
3,Cresselia,Psychic,None,120,70,120,75,130,85
4,Frogadier,Water,None,54,63,52,83,56,97
5,Sceptile,Grass,None,70,110,75,145,85,145
6,Lapras,Water,Ice,130,85,80,85,95,60
7,Sharpedo,Water,Dark,70,140,70,110,65,105
8,Slowbro,Water,Psychic,95,75,180,130,80,30


### WARM-UP
How many mono-type Pokemon are in the database? (i.e. Pokemon without a 2nd type)

In [5]:
query = """
SELECT COUNT(*) FROM pokemon WHERE type_2='None';
"""
pretty_query(cur, query, conn)

,count
0,317


### QUESTION 1
There's one Pokemon in particular that has defined the competitive battling landscape for a dramatically longer amount of time than any other Pokemon this decade. Your first assignment is to identify that Pokemon, which can be defined as the most number of months with #1 usage. I'd like to also see any other Pokemon that was #1 in usage along with how many months they were #1. Your output table should include 2 columns, name & months_on_top ordered by months_on_top descending of course
<br><br>

[Solution](#Q1)

In [6]:
query = """
;
"""
pretty_query(cur, query, conn)

### QUESTION 2
Which Pokemon was the greatest threat to that Pokemon? 
(i.e. the highest average check percentage - only count pokemon who faced off in at least 1,000 battles in any given month and had a check percentage of at least 0.75 for more than 5 months).
<br><br>

[Solution](#Q2)

In [7]:
query = """
;
"""
pretty_query(cur, query, conn)

### QUESTION 3
Although this Pokemon has been force to be reckoned with for half of the last decade, showdown players have demonstrated a fair amount of experimentation to the standard build during it's reign on top. What were it's most commonly used items and natures each month?
<br><br>

[Solution](#Q3)

In [8]:
query = """
;
"""
pretty_query(cur, query, conn)

# RUN THIS CELL WHEN YOU'RE DONE OR ELSE I WILL FIND YOU AND HURT YOU

In [9]:
cur.close()  # Close the cursor
conn.close()

# Q1

In [10]:
query = """
SELECT i.name, COUNT(*) AS months_on_top 
  FROM 
      (
        SELECT item_id,
                RANK() OVER (PARTITION BY month ORDER BY count DESC) r
          FROM battle_items
    ) ranks
  JOIN items i 
    ON ranks.item_id=i.id
 WHERE r=1
 GROUP BY i.name
 ORDER BY months_on_top DESC;
"""
pretty_query(cur, query, conn)

,name,months_on_top
0,leftovers,46
1,toxicorb,5
2,choicescarf,5
3,choicespecs,2
4,heavydutyboots,1
5,blacksludge,1
6,rockyhelmet,1


# Q2

In [11]:
query = """
WITH lando_counters(month, counter_id, check_pct)
AS
    (SELECT c.month, c.counter_id, c.check_pct
      FROM counters c
     JOIN battles b ON c.counter_id=b.id AND c.month=b.month
     WHERE c.id=(SELECT id FROM pokemon WHERE name='Landorus-Therian')
      AND check_pct >=0.75 AND num_battles >=1000 AND b.usage > 0.05),
      
      top_threats(counter_id, check_pct)
AS 
    (SELECT counter_id, AVG(check_pct)
       FROM lando_counters
       WHERE counter_id IN (SELECT counter_id FROM lando_counters GROUP BY counter_id HAVING COUNT(*) > 5)
       GROUP BY counter_id)

SELECT name FROM top_threats
  JOIN pokemon ON top_threats.counter_id=pokemon.id
  WHERE check_pct=(SELECT MAX(check_pct) FROM top_threats);
"""
pretty_query(cur, query, conn)

,name
0,Serperior


# Q3

In [12]:
query = """

WITH lando_nature(name, month, nature, count)
  AS 
	(SELECT p.name, b.month, n.name, bn.count 
      FROM (SELECT * FROM pokemon WHERE name='Landorus-Therian') p 
	  JOIN battles b ON p.id=b.id
	  JOIN battle_natures bn ON b.id=bn.id AND b.month=bn.month
	  JOIN natures n ON bn.nature_id=n.id
      ),

      lando_item(name, month, item, count)
  AS 
	(SELECT p.name, b.month, i.name, bi.count 
      FROM (SELECT * FROM pokemon WHERE name='Landorus-Therian') p 
	  JOIN battles b ON p.id=b.id
	  JOIN battle_items bi ON b.id=bi.id AND b.month=bi.month
	  JOIN items i ON bi.item_id=i.id)

SELECT ln.month,
       ln.nature, ln.count AS nature_count,
       li.item,li.count AS item_count
  FROM (SELECT ln.name, ln.month, ln.nature, ln.count
          FROM lando_nature ln
          WHERE ln.count IN 
		(SELECT MAX(count)
          	FROM lando_nature
	 	GROUP BY month)) ln
  JOIN (SELECT * FROM lando_item li
          WHERE count IN 
                        (SELECT MAX(count) FROM lando_item GROUP BY month)
                        ) li
        
    ON ln.month=li.month
;
"""
pretty_query(cur, query, conn)

,month,nature,nature_count,item,item_count
0,08-2015,Jolly,275210.0,choicescarf,294933.0
1,09-2015,Jolly,236562.0,choicescarf,256053.0
2,10-2015,Jolly,236347.0,choicescarf,262833.0
3,11-2015,Jolly,167246.0,choicescarf,184707.0
4,12-2015,Impish,165979.0,choicescarf,176194.0
5,01-2016,Impish,233486.0,choicescarf,241442.0
6,02-2016,Impish,225733.0,choicescarf,227736.0
7,03-2016,Impish,231734.0,choicescarf,247478.0
8,04-2016,Impish,220810.0,choicescarf,168127.0
9,05-2016,Impish,281200.0,choicescarf,228210.0
